In [2]:
using Oscar
using Combinatorics

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.5.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2021 by The Oscar Development Team


In [24]:
"""
    slicematrix(::AbstractMatrix{<:Number})

    Take a two-dimensional matrix and output a list of its row vectors.

# Examples
```jldoctest makeSmoothWithDependencies
julia> A=[1 2; 3 4]

julia> slicematrix(A)
[[ 1 ,  2 ], [ 3 ,  4 ]]
"""

function slicematrix(A::AbstractMatrix{<:Number})
    return [A[i, :] for i in 1:size(A,1)]
end

"""
    rowMinors(::AbstractMatrix{<:Number},)

    Identical to slicematrix, except only returns row vectors indexed by a set S.

# Examples
```jldoctest makeSmoothWithDependencies
julia> A=[1 2 3;4 5 6; 7 8 9]

julia> S=Set([1,3])

julia> rowMinors(A,S)
2×3 LinearAlgebra.Transpose{Int64,Array{Int64,2}}:
 1  2  3
 7  8  9
"""

function rowMinors(A::AbstractMatrix{<:Number},S)
    outList=[]
    slices=slicematrix(A)
    for i in 1:size(slices,1)
        if i in S
            append!(outList,[slices[i]])
        end
    end
    return transpose(hcat(outList...))
end

"""
    convertIncidenceMatrix(::Polymake.IncidenceMatrixAllocated{Polymake.NonSymmetric})

    Takes a Polymake incidence matrix (e.g., the output of X.MAXIMAL_CONES for a toric variety X) and outputs a list of vectors,
    with each vector recording the indices marked on a given row of the incidence matrix.

# Examples
```jldoctest makeSmoothWithDependencies
julia> X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 0;1 1; 0 1],INPUT_CONES=[[0,1],[1,2]])

julia> M=X.MAXIMAL_CONES

julia> convertIncidenceMatrix(M)
[[ 1 ,  2 ], [ 2 ,  3 ]]
"""

function convertIncidenceMatrix(A::Polymake.IncidenceMatrixAllocated{Polymake.NonSymmetric})
    A=Array(A)
    dim1=size(A,1)
    dim2=size(A,2)
    out=[]
    for i in 1:dim1
        members=[]
        for j in 1:dim2
            if A[i,j]==true
                append!(members,j)
            end
        end
        append!(out,[members])
    end
    return convert.(Array{Int64, 1}, out)
end

"""

    findBarycenter(::AbstractSet,::Polymake.BigObjectAllocated)

    Takes a normal toric variety X and a set s corresponding to a subset of rays of X, and outputs a polymake vector
    corresponding to the barycenter of those rays.

# Examples
```jldoctest makeSmoothWithDependencies
julia> X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 0;1 1; 0 1],INPUT_CONES=[[0,1],[1,2]])

julia> s=[1,2]

julia> findBarycenter(s,X)
pm::Matrix<pm::Integer>
2 1

"""

function findBarycenter(s,X::Polymake.BigObjectAllocated)
    rays = rowMinors(Array(X.RAYS), s)
    dim=size(rays,2)
    bary=zeros(Polymake.Rational,dim,1)
    for i in 1:size(rays,1)
        bary+=rays[i,:]
    end
    bary=Polymake.common.primitive(transpose(bary))
    return bary
end

function toric_blowup(s, X, v)
    s = [i + 1 for i in s]
    if v==nothing
        v=findBarycenter(s,X)
    end
    coneList = convertIncidenceMatrix(X.MAXIMAL_CONES)
    starIndex = findall((t) -> all(((i) -> i in t).(s)), coneList)
    star = [coneList[i] for i in starIndex]
    rayMatrix = X.RAYS
    
    lattice = X.HASSE_DIAGRAM
    faces = @Polymake.convert_to Array{Set{Int}} lattice.FACES
    
    clStar = []
    for t in star
        c = rank(Array(rowMinors(rayMatrix, t))) - 1
        rank_c_subcone_indices = @Polymake.convert_to Array{Int} Polymake.graph.nodes_of_rank(lattice,c)
        rank_c_subcones = [faces[i + 1] for i in rank_c_subcone_indices]
        for cone in rank_c_subcones
            new_cone = [i+1 for i in cone]
            if all((i -> i in t).(new_cone))
                push!(clStar, new_cone)
            end
        end
    end
    clStar = unique(clStar)
    
    n = size(rayMatrix, 1) + 1
    coneList = filter(x -> !(x in star), coneList)
    
    if length(s) == 1
        newCones = []
        for t in clStar
            if !(s[1] in t)
                push!(newCones, sort(push!(t, s[1])))
            end
        end
        finalCones = [[i - 1 for i in cone] for cone in append!(coneList, newCones)]
        return Polymake.fulton.NormalToricVariety(INPUT_RAYS = Array(X.RAYS), INPUT_CONES = append!(coneList, newCones))
    end
    newCones = []
    for t in clStar
        if any(((i) -> !(i in t)).(s))
            push!(newCones, push!(t, n))
        end
    end
    finalRays = vcat((X.RAYS),v)
    finalCones = [[i - 1 for i in cone] for cone in append!(coneList, newCones)]
    return Polymake.fulton.NormalToricVariety(INPUT_RAYS = finalRays, INPUT_CONES = finalCones)
end

"""

    convertBool(::AbstractVector)

    Takes a column vector of boolean values and converts it to a vector of indices marked 'true'.

#Examples
```jldoctest makeSmoothWithDependencies
julia> B=[true true false true]

julia> convertBool(transpose(B))
[0, 1, 3]
"""

function convertBool(B::AbstractVector)
    out=[]
    for i in 1:size(B,1)
        if B[i]==true
           append!(out,i-1) 
        end
    end
    return out
end

convertBool (generic function with 1 method)

In [52]:
"""

    getConeRank(::AbstractMatrix,::AbstractVector)

    Takes a matrix and a vector containing indices corresponding to rows of a matrix,
    and calculates the rank of the matrix consisting only of those rows.

#Examples
```jldoctest makeSmoothWithDependencies
julia> v=[1,2]

julia> M=[0 1; 1 1; 1 0]

julia> getConeRank(v,M)
2
"""

function getConeRank(coneRayIndices::AbstractVector, rayMatrix::AbstractMatrix)
    coneRays = rowMinors(rayMatrix,coneRayIndices)
    return rank(Matrix(coneRays))
end

"""
    getDimension(::Polymake.BigObjectAllocated)

    Returns the ambient dimension of a normal toric variety.

#Examples
```jldoctest makeSmoothWithDependencies
julia> X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 0 0;1 2 0;0 0 1;0 1 0; 1 1 1],INPUT_CONES=[[0,1,2],[0,2,3,4]])

julia> getDimension(X)
3

"""

function getDimension(X)
    return size(X.RAYS, 2)
end



function getConeFacesImproved(fan,cone,rayMatrix)
    lattice = fan.HASSE_DIAGRAM
    faces = @Polymake.convert_to Array{Set{Int}} lattice.FACES
    cone_faces=[]
    c = rank(Array(rowMinors(rayMatrix, cone))) - 1
    rank_c_subcone_indices = @Polymake.convert_to Array{Int} Polymake.graph.nodes_of_rank(lattice,c)
    rank_c_subcones = [faces[i + 1] for i in rank_c_subcone_indices]
    for subcone in rank_c_subcones
        new_cone = [i+1 for i in subcone]
        if all((i -> i in cone).(new_cone))
            push!(cone_faces, new_cone)
        end
    end 
    return cone_faces
end

function makeSimplicial(X)
    Y = copy(X)
    while (true)
        print(Y.RAYS)
        print(Y.MAXIMAL_CONES)
        if Y.SIMPLICIAL==true
            break
        end
        #Maximal cones and ray matrix
        coneList = Y.MAXIMAL_CONES
        rayMatrix = Y.RAYS
        badCone = nothing
        for i in 1:size(coneList,1)
            cone = coneList[i,:]
            if (getConeRank(cone, rayMatrix) != size(cone))
                badCone = cone
            end
        end
        if (badCone == nothing)
            # All cones are linearly independent
            break
        else
            # Find the first ray that is contained in more than one orbit
            # and subdivide at that ray, using toricBlowup
            
            # Get faces (need to replace this)
            edges = getConeFacesImproved(Y,badCone,rayMatrix)
            # Find the first ray that is contained in more than one orbit
            i = 1
            while count(r->(badCone[i] in r), edges) == 1
                i += 1
            end
            # Subdivide at the cone containing just that ray
            Y = toric_blowup(convertBool(badCone), Y,nothing)
            #Y = toric_blowup([badCone[i]], Y,nothing)
        end
        # Repeat this process until there are no more bad cones
        #break
    end
    return Y
end



makeSimplicial (generic function with 1 method)

In [53]:
function coneListFormat(coneList)
    memberList=[]
    denseList=slicematrix(coneList)
    for row in denseList
        members=[]
        for i in 1:size(row,1)
            if row[i]==1
                append!(members,i)
            end
        end
        append!(memberList,[members])
    end
    return memberList
end
 
function makeSmooth(X)
    Y  = copy(X)
    while(true)
        coneList = convertIncidenceMatrix(Y.MAXIMAL_CONES)
        rayMatrix = Array(Y.RAYS)
        
        k = 1
        # Iterate through the coneList, getting the index of the first cone not smooth
        for coneSet in coneList
            # Getting the number of rays in coneSet
            S=size(coneSet)[1]
            coneRays=rowMinors(rayMatrix,coneSet)
            # Checking whether this cone is smooth
            smoothCheck=Polymake.fan.check_fan_objects(Polymake.polytope.Cone(RAYS=coneRays)).SMOOTH_FAN
            if !smoothCheck
                # If the cone is not simplicial or not smooth, we have found the cone that we need to make smooth
                break
            else
                k+=1
            end
        end
        # At this point, all the cones are smooth. The program terminates.
        if k == size(coneList,1)+1
            break
        end
        
        # Get the cone that we found to be not smooth
        sigma=coneList[k]
        sigmaRays=slicematrix(rowMinors(rayMatrix,sigma))
        tau=0; tauRays=0; tauCone=0
        # Iterate over the subcones of sigma, finding tau, the smallest one that is not smooth
        for subset in collect(powerset(sigma))
            if size(subset,1) > 1
                S=size(subset)[1]
                subsetRays=rowMinors(rayMatrix,subset)
                subsetCone=Polymake.polytope.Cone(RAYS=subsetRays)
                smoothCheck=Polymake.fan.check_fan_objects(subsetCone).SMOOTH_FAN
                if !smoothCheck
                    tau=subset
                    tauRays=subsetRays
                    tauCone=subsetCone
                    break
                end 
            end
        end
        
        # Getting the Hilbert Basis of tau
        H=slicematrix(Matrix(tauCone.HILBERT_BASIS_GENERATORS[1]))
        rayIndex=0
        # Iterate over the Hilbert Basis, finding the first ray that is not the generator of sigma
        for i in 1:size(H,1)
            if !(H[i] in sigmaRays)
                rayIndex=i
                break
            end
        end
        if rayIndex==0
            # Every Hilbert Basis of tau is a generator of sigma. Make Y simplicial is sufficient to make sigma smooth
            #return Y
            Y=makeSimplicial(Y)
            break
        else
            # blowupRay is not a generator of sigma, blow up tau at blowupRay
            blowupRay=H[rayIndex]
            tau=[i-1 for i in tau]
            Y=toric_blowup(tau,Y,blowupRay)
            break
        end
    end
    return Y
end

makeSmooth (generic function with 1 method)

In [54]:
X=Polymake.fulton.NormalToricVariety(INPUT_RAYS=[1 0 0;1 1 0;1 0 1;1 1 1;1 -2 0],INPUT_CONES=[[0,1,2,3],[0,4]])

type: NormalToricVariety

INPUT_CONES
	{0 1 2 3}
	{0 4}

INPUT_RAYS
	1 0 0
	1 1 0
	1 0 1
	1 1 1
	1 -2 0

In [55]:
makeSmooth(X)

pm::Matrix<pm::Rational>
1 0 0
1 1 0
1 0 1
1 1 1
1 -2 0
pm::IncidenceMatrix<pm::NonSymmetric>
{0 1 2 3}
{0 4}
pm::Matrix<pm::Rational>
1 0 0
1 1 0
1 0 1
1 1 1
1 -1 0
1 -2 0
pm::IncidenceMatrix<pm::NonSymmetric>
{0 1 2 3}
{0 4}
{4 5}
pm::Matrix<pm::Rational>
1 0 0
1 1 0
1 0 1
1 1 1
1 -1 0
1 -3/2 0
1 -2 0
pm::IncidenceMatrix<pm::NonSymmetric>
{0 1 2 3}
{0 4}
{4 5}
{5 6}
pm::Matrix<pm::Rational>
1 0 0
1 1 0
1 0 1
1 1 1
1 -1 0
1 -3/2 0
1 -7/4 0
1 -2 0
pm::IncidenceMatrix<pm::NonSymmetric>
{0 1 2 3}
{0 4}
{4 5}
{5 6}
{6 7}
pm::Matrix<pm::Rational>
1 0 0
1 1 0
1 0 1
1 1 1
1 -1 0
1 -3/2 0
1 -7/4 0
1 -15/8 0
1 -2 0
pm::IncidenceMatrix<pm::NonSymmetric>
{0 1 2 3}
{0 4}
{4 5}
{5 6}
{6 7}
{7 8}
pm::Matrix<pm::Rational>
1 0 0
1 1 0
1 0 1
1 1 1
1 -1 0
1 -3/2 0
1 -7/4 0
1 -15/8 0
1 -31/16 0
1 -2 0
pm::IncidenceMatrix<pm::NonSymmetric>
{0 1 2 3}
{0 4}
{4 5}
{5 6}
{6 7}
{7 8}
{8 9}
pm::Matrix<pm::Rational>
1 0 0
1 1 0
1 0 1
1 1 1
1 -1 0
1 -3/2 0
1 -7/4 0
1 -15/8 0
1 -31/16 0
1 -63/32 0
1 -2 0
pm::Inci

LoadError: [91mException occured at Polymake side:[39m
[91minitial check failed: [39m
[91minvalid input: number of rays used in INPUT_CONES exceeds number of INPUT_RAYS at /home/kitty/.julia/artifacts/c2e908f420af909af6125716f044b1f7635ee267/share/polymake/apps/fan/rules/initial.rules line 40.[39m
[91m	Polymake::fan::PolyhedralFan::__prod__aac217(Polymake::fulton::NormalToricVariety=ARRAY(0x1faf5608)) called at /home/kitty/.julia/artifacts/c2e908f420af909af6125716f044b1f7635ee267/share/polymake/perllib/Polymake/Core/Rule.pm line 820[39m
[91m	eval {...} called at /home/kitty/.julia/artifacts/c2e908f420af909af6125716f044b1f7635ee267/share/polymake/perllib/Polymake/Core/Rule.pm line 812[39m
[91m	Polymake::Core::Rule::execute_me called at /home/kitty/.julia/artifacts/c2e908f420af909af6125716f044b1f7635ee267/share/polymake/perllib/Polymake/Core/Rule.pm line 805[39m
[91m	Polymake::Core::Rule::execute(Polymake::Core::Rule=ARRAY(0x161ded90), Polymake::fulton::NormalToricVariety=AR